In [103]:
## data format
import pandas as pd
import numpy as np
import polars as pl
import pickle

## preprocessing features
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import string
from nltk.corpus import stopwords

## predictor
from catboost import CatBoostRegressor

## garbage collection
import gc

`-` 정리

In [105]:
with open('trained_model.pkl', 'wb') as f :
    pickle.dump(predictr, f)

In [106]:
with open('transformr.pkl', 'wb') as f :
    pickle.dump(pipes, f)

In [ ]:
import os
import polars as pl
import numpy as np
import pandas as pd
import catboost
import kaggle_evaluation.mcts_inference_server

target = 'utility_agent1'
counter = 0  # 첫 번째 호출 확인용 변수

features_list = [12, 21, 28, 30, 39, 40, 41, 42, 43, 44, 45, 47, 49, 51, 52,
                 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
                 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
                 88, 89, 91, 92, 94, 95, 96] + list(range(97, 121))

def train_model():
    global predictr
    df_train = pd.read_csv("/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv")
    
    X = df_train.iloc[:, [1,2,3]+features_list+list(range(121, 810))]
    y = df_train[target]

    predictr = catboost.CatBoostRegressor(cat_features=["GameRulesetName", "agent1", "agent2", "EnglishRules", "LudRules"])
    predictr.fit(X, y)

def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    global counter
    if counter == 0:  # 첫 번째 호출 시에만 모델을 학습하거나 로드
        train_model()
        counter += 1
    
    predictions = pd.Series(predictr.predict(test.to_pandas().iloc[:, [1,2,3]+features_list+list(range(121, 810))])).map(lambda x : x if (x <= 1) & (x >= -1) else 1 if x > 1 else -1)
    return sample_sub.with_columns(pl.Series('utility_agent1', predictions))

# 인퍼런스 서버 설정
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )

In [3]:
import os
import polars as pl
import numpy as np
import pandas as pd
import catboost
import kaggle_evaluation.mcts_inference_server

target = 'utility_agent1'
counter = 0  # 첫 번째 호출 확인용 변수

features_list = [12, 21, 28, 30, 39, 40, 41, 42, 43, 44, 45, 47, 49, 51, 52,
                 55, 56, 57, 59, 61, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
                 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87,
                 88, 89, 91, 92, 94, 95, 96] + list(range(97, 121))

def train_model():
    global predictr
    df_train = pd.read_csv("/root/kag_comp/UM_MCTS/Data/train.csv")
    
    X = df_train.iloc[:, [1,2,3]+features_list+list(range(121, 810))]
    y = df_train[target]

    predictr = catboost.CatBoostRegressor(cat_features=["GameRulesetName", "agent1", "agent2", "EnglishRules", "LudRules"])
    predictr.fit(X, y)

def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    global counter
    if counter == 0:  # 첫 번째 호출 시에만 모델을 학습하거나 로드
        train_model()
        counter += 1
    
    predictions = pd.Series(predictr.predict(test.to_pandas().iloc[:, [1,2,3]+features_list+list(range(121, 810))])).map(lambda x : x if (x <= 1) & (x >= -1) else 1 if x > 1 else -1)
    return sample_sub.with_columns(pl.Series('utility_agent1', predictions))

# 인퍼런스 서버 설정
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/root/kag_comp/UM_MCTS/Data/test.csv',
            '/root/kag_comp/UM_MCTS/Data/sample_submission.csv'
        )
)

In [ ]:
## data format
import pandas as pd
import numpy as np
import polars as pl
import pickle

## preprocessing features
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import string
from nltk.corpus import stopwords

## predictor
from catboost import CatBoostRegressor

## evaluation server
import kaggle_evaluation.mcts_inference_server

target = 'utility_agent1'
counter = 0  # 첫 번째 호출 확인용 변수

def train_model():
    global predictr
    df_train = pd.read_csv("/root/kag_comp/UM_MCTS/Data/train.csv")
    
    X = df_train.drop(["num_wins_agent1", "num_draws_agent1", "num_losses_agent1", target], axis = 1)
    y = df_train[target]

    predictr = catboost.CatBoostRegressor(cat_features=["GameRulesetName", "agent1", "agent2"], verbose = 0)
    predictr.fit(X, y)

def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    global counter
    if counter == 0:  # 첫 번째 호출 시에만 모델을 학습하거나 로드
        train_model()
        counter += 1
    
    predictions = predictr.predict(test.to_pandas())
    return sample_sub.with_columns(pl.Series('utility_agent1', predictions))

# 인퍼런스 서버 설정
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/root/kag_comp/UM_MCTS/Data/test.csv',
            '/root/kag_comp/UM_MCTS/Data/sample_submission.csv'
        )
)

1. 하이퍼볼릭 탄젠트 함수로 타겟 값을 변환 / 예측 시 역함수로 반환

> CatBoost는 변수가 의미하는 값을 직접 학습할 때 성능이 좋으므로, 변환된 데이터를 사용한다면 성능 저하가 발생할 수 있음.

In [14]:
# X = df_train.drop(["num_wins_agent1", "num_draws_agent1", "num_losses_agent1","utility_agent1"])
# y = df_train['utility_agent1'].map_elements(lambda x : 1/2*np.log((1+x)/(1-x)) if (x > -0.99999) & (x < 0.99999) else 1/2*np.log(0.00000001/2) if x > -0.99999 else 1/2*np.log(1.99999999/0.00000001))

# predictr = CatBoostRegressor(cat_features = ["GameRulesetName", "agent1", "agent2", "EnglishRules", "LudRules"])
# predictr.fit(X.to_pandas(), y.to_pandas())
# pd.Series(predictr.predict(XX)).map(lambda x : (np.exp(2*x) - 1)/(np.exp(2*x)+1))

2. 자연어 TF-IDF로 변환

In [68]:
## data
nl_cols = ["EnglishRules", "LudRules"]
NL_train = df_train[nl_cols].astype(str).values.ravel()

## pipelines
pipes = [Pipeline([('bow', CountVectorizer()), ('tfid', TfidfTransformer())]) for i in nl_cols]
# Bag of Words format / TF-IDF format(Final Form)

## fitting transformer
X_NL_list = []

for i, col in enumerate(nl_cols) :
    X_NL_col = pipes[i].fit_transform(NL_train[col])
    X_NL_list.append(X_NL_col.toarray())

## transform
X_NL = pd.DataFrame(np.hstack(X_NL_list))

In [90]:
targets = ['num_wins_agent1', 'num_draws_agent1', 'num_losses_agent1', 'utility_agent1']
target = 'utility_agent1'

X = pd.concat([df_train.drop(targets+nl_cols, axis = 1), X_NL], axis = 1)
y = df_train[target]

In [51]:
NL_test = df_test[nl_cols]
XX_NL_list = []

for i, col in enumerate(nl_cols) :
    XX_NL_col = pipes[i].transform(NL_test[col])
    XX_NL_list.append(XX_NL_col.toarray())

XX_NL = pd.DataFrame(np.hstack(XX_NL_list))

In [94]:
yhat = pd.Series(predictr.predict(X)).map(lambda x : x if (x <= 1) & (x >= -1) else 1 if x > 1 else -1)

In [101]:
np.mean(abs(y - yhat))

0.1873588177652906

In [93]:
predictr = CatBoostRegressor(cat_features = ["GameRulesetName", "agent1", "agent2"])
predictr.fit(X, y)

Learning rate set to 0.096893
0:	learn: 0.5975717	total: 285ms	remaining: 4m 44s
1:	learn: 0.5768303	total: 556ms	remaining: 4m 37s
2:	learn: 0.5587568	total: 682ms	remaining: 3m 46s
3:	learn: 0.5426684	total: 892ms	remaining: 3m 42s
4:	learn: 0.5293401	total: 1.09s	remaining: 3m 37s
5:	learn: 0.5170463	total: 1.39s	remaining: 3m 50s
6:	learn: 0.5070711	total: 1.58s	remaining: 3m 43s
7:	learn: 0.4978081	total: 1.85s	remaining: 3m 49s
8:	learn: 0.4904947	total: 1.97s	remaining: 3m 37s
9:	learn: 0.4837217	total: 2.25s	remaining: 3m 43s
10:	learn: 0.4775586	total: 2.48s	remaining: 3m 42s
11:	learn: 0.4724841	total: 2.65s	remaining: 3m 38s
12:	learn: 0.4675885	total: 2.77s	remaining: 3m 30s
13:	learn: 0.4632958	total: 3.05s	remaining: 3m 34s
14:	learn: 0.4592712	total: 3.26s	remaining: 3m 34s
15:	learn: 0.4565762	total: 3.48s	remaining: 3m 33s
16:	learn: 0.4520563	total: 3.85s	remaining: 3m 42s
17:	learn: 0.4480722	total: 4.06s	remaining: 3m 41s
18:	learn: 0.4432874	total: 4.39s	remaining: